# Sorcero Test
## Tanmayan Pande
## Text Summarization

The following block contains the import statements used in the assignment

In [133]:
import numpy as np
import os
from tqdm import tqdm, trange
import string
import re

Stop Words obtained from: https://gist.github.com/sebleier/554280

### Implementation

The basic idea behind this text summarizer is that we can generate sentence embeddings by taking a weighted sum of the glove word embeddings of each word in the input text file. Since we have just a single, small text file, I am only taking the frequency with which each word occurs, after removing the stop words in the text file, as the weight of that word. For text corpuses with multiple documents, TF-IDF weights might be a better solution.

The algorithm is as follows:
1. Strip each sentence of punctuations and convert it into lowercase
2. For each word in the sentence, get the embeddings of the word according to pre-trained glove embeddings
3. Calculate the normalized frequency of the word. The normalized frequency is defined as the frequency of the word divided by the frequency of the word which occurs most in the text document.
4. Calculate the sentence embedding by using the formula \begin{equation*} weight(sent) = \sum_{i = 1}^n freq_{norm}(word) \times embedding(word) \end{equation*} where $n$ is the number of words in that sentence.
5. Based on what percent of the text we want as a summary, we calculate the number of clusters that we need to divide the text into.
6. We then fit a K-Means clustering model based on the number of clusters thus calculated and the sentence embeddings we calculated earlier
7. Based on the means of the K-Means clustering, we find the closest sentence in each cluster and return the sentence.
8. Then, we order these sentences based on the sentence order in the original text and return the summary

#### Class ```KMeans```:

An implementation of the KMeans clustering algorithm

##### Constructor:
Variables:
1. ```data``` : The data variable which contains the data to be clustered
2. ```clusters``` : The number of clusters to be fitted

Description: Initalizes a KMeans model

##### ```fit()```
Variables:
None

Description: Fits the KMeans model on the data. This will run till the means do not change for one iteration
Returns: The cluster means

##### ```Euclidean_dist(point, array)```
Variables:
1. ```point``` : The first set of points
2. ```array``` : The second set of points

Description: Finds the Euclidean Distance between each row of two matrices
Returns: The Euclidean distance between the point and the array

##### ```get_closest_point(means)```
Variables:
1. ```means```: The means of the clusters in the KMeans model

Description: Returns the point closest to the mean in each cluster
Returns : The point closest to the mean in each cluster

In [134]:
class KMeans():
    def __init__(self, data, clusters):
        self.data = data
        self.clusters = clusters

    def Euclidean_dist(self, point, array):
        
        # new_mat is the difference between the point and the array
        new_mat = array - point
        dist_mat = np.zeros(new_mat.shape[0])
        
        # For each point in the set of points, calculate te distance
        # of the point from all points in the data matrix
        for i in range(0, new_mat.shape[0]):
            dist_mat[i] = np.matmul(new_mat[i, :].T, new_mat[i, :])
        
        # Return the distance matrix
        return dist_mat

    def fit(self):
        
        # Declare the storage variables and initialize the means using a random distribution
        dims = self.data.shape[1]
        means = np.zeros((self.clusters, dims))
        new_means = np.mean(self.data, axis = 0) + np.random.randn(self.clusters, dims) * np.std(self.data, axis = 0)
        
        # While the current means and new means are not the same, do
        while np.all(Euclidean_dist(means, new_means) != 0):
            means = new_means
            clusters_ = [[] for i in range(self.clusters)]
            
            # Calculate the distance of the point from the cluster mean for all clusters
            dist_mat = np.zeros((self.clusters, self.data.shape[0]))
            for i in range(0, self.clusters):
                dist_mat[i, :] = Euclidean_dist(means[i, :], self.data)
                
            # Get the cluster for which the distance is minimum
            min_index = np.argmin(dist_mat, axis = 0)
            
            # Rearrange the clusters
            for i in range(min_index.shape[0]):
                clusters_[min_index[i]].append(self.data[i])
            
            # Calculate the new means based on the clusters
            for i in range(self.clusters):
                if len(clusters_[i]) != 0:
                    new_means[i, :] = np.mean(clusters_[i], axis = 0)
                else:
                    continue
        
        # Return the means
        return new_means
    
    def get_closest_point(self, means):
        index = []
        
        # For all points in the data, check the point with minimum distance from
        # each cluster mean
        for i in range(0, means.shape[0]):
            dist_mat = Euclidean_dist(self.data, means[i, :])
            index.append(int(np.argmin(dist_mat)))
        
        # Return the indices of these points
        return np.asarray(index, dtype = 'int32')


#### Class ```Summarizer```:

##### Constructor:
Variables:
1. ```text_path``` : The path of the text file to be summarized
2. ```embedding_dims``` : The embedding size for the words. Valid values are 50, 100, 200, 300
3. ```stop_words``` : Boolean flag indicating whether to keep stop words, or remove them. ```True``` signifies remove stop words
4. ```summary_percent``` : The percent reduction in the text required. If between 0 and 1, the number is assumed to be a fraction of the summary. If greater than 1, the number is assumed to be the number of words in the summary.

##### ```gen_embeddings()```:
Variables:
None

Description: Constructs the word embeddings based on the pre-trained glove embeddings

##### ```format_text()```:
Variables:
None

Description: Formats the text as per the requirements of the model (converts to lowercase, strips punctuation)
Returns : The formatted text

##### ```gen_count(input_lines)```:

Variables:
1. ```input_lines``` : The set of sentences formatted according to the requirements of the model

Description: Generates the word frequency dictionary and the final input sentence combination

##### ```get_sentence_embeddings()```:
Variables:
None

Description: Generates the sentence embeddings according to the formula given above
Returns: The sentence embeddings

##### ```get_sentences()``` :
Variables:
None

Description: Generates the final summary by fitting a KMeans model on the sentence embeddings from the previous model
Returns: The summarized version of the text

##### ```Summarize()``` :
Variables:
None

Description: Wrapper function for all the above functionality
Returns: The summarized version of the text

In [135]:
class Summarizer():
    def __init__(self, text_path, embedding_dims = 50, stop_words = True, summary_percent = 0.2):
        self.embedding_dims = embedding_dims
        src = os.getcwd()
        
        # Open the input text and read the input
        with open(src + "/" + text_path, "r", encoding = 'utf8') as file:
            self.input_text = file.readline()
        
        # If stop words are to be removed, open and store the stop words
        if stop_words:
            with open(src + "/stop_words.txt", "r", encoding = 'utf8') as file:
                self.stop_words = file.readlines()
        self.stop_flag = stop_words
        n_words = self.input_text.split(" ")
        
        # Check the summary percent. It can't be negative or greater than the number of 
        # words in the text (n_words). If the summary percent is a number between 1 and 
        # n_words, it is converted into a fraction by dividing by the number of words 
        # in the text
        if summary_percent < 0:
            raise ValueError("The fraction of text cannot be a negative value")
        elif summary_percent > 0 and summary_percent <= 1:
            self.percent_summary = summary_percent
        elif summary_percent > 1 and summary_percent <= n_words: 
            self.percent_summary = summary_percent / n_words
        else:
            raise ValueError("The number of words in the summary can't be greater than the number of words in the text")

    def gen_embeddings (self):
        self.embeddings = {}
        
        # Open the pre-trained embeddings and store them in a dictionary for easy access
        with open(src + "/glove.6B.{}d.txt".format(self.embedding_dims), "r", encoding = 'utf8') as file:
            lines = file.readlines()
            for line in tqdm(lines, desc = "Loading Embeddings..."):
                line = line.strip()
                split_ = line.split(" ")
                self.embeddings[split_[0]] = np.asarray(split_[1:]).astype('float32')

    def format_text(self):
        if self.stop_flag:
            self.stop_words = [stop_word.strip() for stop_word in self.stop_words]
            
        # We use a regex to split the sentences, as there might be abbreviations in the text 
        # which might be unnecessarily split is we use the string.split method
        pat = r'\.\s'
        self.input_text = re.split(pat, self.input_text)
        
        # We then strip each line for empty spaces at the start and end,
        # convert each line to lower case and 
        # strip any punctuations from the line
        input_lines = [line.strip() for line in self.input_text]
        input_lines = [line.lower() for line in input_lines]
        input_lines = [line.translate(line.maketrans("", "", string.punctuation)) for line in input_lines]
        return input_lines
    
    def gen_count(self, input_lines):
        self.word_count = {}
        self.rectified_input = []
        
        # For each line in the input line,
        for line in input_lines:
            
            # We split the line into words
            for word in line.split(" "):
                
                # If we are removing stop words, 
                # ignore such words in the text
                if self.stop_flag:
                    if word in stop_words:
                        continue
                    # If the word is already in the
                    # dictionary, increase its count
                    if word in self.word_count:
                        self.word_count[word] += 1
                    
                    # Else, add the word in the dictionary
                    else:
                        self.word_count[word] = 1
                else:
                    if word in self.word_count:
                        self.word_count[word] += 1
                    else:
                        self.word_count[word] = 1
            self.rectified_input.append(line)
    
    def get_sentence_embeddings(self):
        
        # Get the word which occurs most often in the text
        max_freq = max(list(self.word_count.values()))
        
        # Normalize the word frequency by dividing by the max frequency
        self.word_count = { word : values / max_freq 
                      for word, values in zip(list(self.word_count.keys()), list(self.word_count.values())) }

        sent_weights = {}
        
        # For every sentence in the rectified input
        for line in self.rectified_input:
            
            # The sentence embedding is a vector of zeroes initially
            weight = np.zeros(self.embedding_dims)
            
            # For each word in the sentence
            for word in line.split(" "):
                try:
                    
                    # Add the weighted embedding of the word to the sentence 
                    # embeddings
                    weight += self.word_count[word] * self.embeddings[word]
                except:
                    
                    # If the word is not in the dictionary (i.e. it is a stop word),
                    # ignore it
                    continue
            
            
            # Normalize the sentence embedding by the length of the sentence.
            # This is to ensure that longer sentences are not preferred
            weight = weight / len(line.split(" "))
            sent_weights[self.rectified_input.index(line)] = weight
        
        # Convert the dictionary of sentence embeddings into an array
        # and return it
        sent_weights_array = np.asarray([value for value in list(sent_weights.values())])
        return sent_weights_array
    
    def get_sentences(self, sent_weights):
        percent_summary = self.percent_summary
        
        # Based on the percent reduction, get the number of clusters
        num_sent = int(percent_summary * len(sent_weights))
        
        # Declare a KMeans model and initialize it
        model = KMeans(sent_weights, num_sent)
        clusters = model.fit()
        
        # Get the sentences closest to the cluster means
        indices = model.get_closest_point(clusters)
        
        # Sort these indices so as to maintain a coherent meaning
        indices.sort()
        
        # Get the actual sentences from the original text
        summary = [self.input_text[index] for index in indices]
        
        # Return the summary
        return ". ".join(summary)
    
    def Summarize(self):
        
        # Just a wrapper function with the flow of the model
        self.gen_embeddings()
        input_lines = self.format_text()
        self.gen_count(input_lines)
        sent_weights = self.get_sentence_embeddings()
        summary = self.get_sentences(sent_weights)
        return summary

In [136]:
summarizer = Summarizer("input.txt", embedding_dims = 50, summary_percent = 0.2)
print(summarizer.Summarize())

Loading Embeddings...: 100%|████████████████████████████████████████████████| 400001/400001 [00:17<00:00, 22241.88it/s]


After creating the Mark II armor, Stark uploaded JARVIS into all of the Iron Man Armors, as well as allowing him to interact with the other Avengers, giving them valuable information during combat. JARVIS' duties were then taken over by FRIDAY.


### Weaknesses and Possible Solutions:

1. The model only picks sentences from the original text in the summary. As such, the summary might not be a great representative of the ideas in the original text.
2. We don't have a lot of freedom in number of words in the summary.

The solution to these problems is to build a deep learning based text summarization tool. Using encoder decoder RNNs (or LSTMs), we can generate shorter versions of the text, which better capture the essence of the text. This is albeit a supervised technique, and hence will require a huge amount of training data to generate good results. This was the only reason I did not choose to build a deep learning based model, instead preferring a simple unsupervised model.